# PR Plots

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')
from plotutils import plot_ts, plot_cal
import wodupcrawler
import os
import numpy as np
from bokeh.models import Div, Panel, Tabs, Select, CustomJS, Column, Slider
from bokeh.io import output_notebook, show
import fitetl, wodupcrawler
import glob
import pandas as pd
from datetime import datetime

output_notebook()

/Users/hasannagib/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Loading BokehJS ...

In [3]:
# WOD logs
#wodupcrawler.main()
wods, df_wods = wodupcrawler.read_wods_json(f'{wodupcrawler.datadir}session_wods.json')
latest_wodup_log_dt = wodupcrawler.get_latest_wodup_log_date(wods)

In [4]:
# PR data
df_pr = pd.read_csv(
    '../../WodUp-Scraper/data/hasannagib-pr-table.csv', 
    parse_dates=[f'date_{mvmt}' for mvmt in ['back_squat', 'front_squat', 'deadlift', 'shoulder_press']]
).query('(reps > 0) and (reps <= 10)')

In [5]:
movements = ['deadlift', 'barbell_bench_press', 'back_squat', 'shoulder_press']
three_lift_total = int(df_pr.query("reps==1")[['barbell_bench_press', 'back_squat', 'deadlift']].sum().sum())


In [6]:

plot_rep_prs, plot_rep_prs_cds = plot_ts(
    df_pr,
    ys=movements,
    hover_vars=[f'date_{mvmt}' for mvmt in movements],
    xvar='reps',
    styles=['-o'],
    x_axis_type='linear',
    ylabel='Weight (lbs)',
    xlabel='Reps',
    title=f'Rep PRs - Three lift total: {three_lift_total} lbs',
    plot_height=350,#275,
    plot_width=900,
    show_plot=False,
    tools='box_zoom,undo,redo,reset',
    palette=['#154ba6', '#3f8dff', '#7ec4ff', 'grey'],
    legend_position='right',
    legend_location='top_right',
    legend_orientation='vertical',

)

tabs = []
df_pr_hist = {}

for i in [1,2,3,4,5]:

    pr_hist = []
    for movement in movements:
        df_hist = pd.read_csv(f'../../WodUp-Scraper/data/hasannagib-{movement.replace("_", "-")}.csv', parse_dates=['date'])
        df = df_hist.query(f'(reps>={i})').sort_values('date')
        dfi = np.maximum.accumulate(df).set_index('date')[['weights']].rename(
            columns={'weights': movement}).sort_index().drop_duplicates().groupby('date').max()
        dfi.index.name='date'
        pr_hist.append(dfi)
    
    plot_pr_hist = pd.concat(pr_hist).dropna(thresh=1).sort_index().fillna(method='ffill').groupby('date').max()
    df_pr_hist[i] = plot_pr_hist
    add = plot_pr_hist.iloc[-1,:]
    add.name = datetime.today().strftime('%Y-%m-%d')
    plot_pr_hist = plot_pr_hist.append(add)

    p, _ = plot_ts(
        plot_pr_hist,
        xvar='date',
        styles=['oL'],
        units=['lbs'],
        x_axis_type='datetime',
        title=f'{i} rep max PR over time ',
        xlabel='Date',
        ylabel='Weight (lbs)',
        plot_height=350,
        plot_width=900,
        tools='box_zoom,undo,redo,reset',
        palette=['#154ba6', '#3f8dff', '#7ec4ff', 'grey'], #e73360
        show_plot=False,
        legend_position='right',
        legend_location='top_right',
        legend_orientation='vertical',

    );

    tabs.append(Panel(child=p, title=f"{i} RM"))

plot_pr_history_tabs = Tabs(tabs=tabs, tabs_location='above', margin=(0,0,0,0))

In [7]:
show(plot_pr_history_tabs)

In [8]:
for i, df in df_pr_hist.items():
    df_pr_hist[i]['Y'] = df[df.columns[0]]

df_pr_cal = {}
plot_pr_cal = {}
cds_pr_cal = {}

for mvmt in movements + ['Y']:
    df_pr_cal[mvmt] = {}
    plot_pr_cal[mvmt] = {}
    cds_pr_cal[mvmt] = {}
    
    for i in [1,2,3,4,5]:
        df = df_pr_hist[i][[mvmt]].reset_index().groupby(mvmt).min().reset_index().set_index('date')
        df = df.reindex(pd.date_range('2019-09-16', datetime.today()))
        df.index.name = 'date'
        df = df.reset_index().rename(columns={mvmt:'weight'})
        
        p, p_cds = plot_cal(
            df, 
            date_column='date', 
            color_column='weight',
            palette=['red'],
            mode='github', 
            fig_args={
                'plot_width':900,
                'plot_height':125,
                'tools':'hover',
                'toolbar_location':None,
                'title':'PR Calendar',
                'x_axis_location':"above"
            }, 
            rect_args={
                'width':1,
                'height':1,
                'line_color':'grey',
                'line_width':1,
            },
            hover_tooltips=[('Weight',f'@weight lbs')], 
            show_dates=False
        )
        
        df_pr_cal[mvmt][i] = df
        plot_pr_cal[mvmt][i] = p
        cds_pr_cal[mvmt][i] = p_cds

In [9]:
select = Select(title="Movement", value="deadlift", options=movements, width=200)
slider = Slider(start=1, end=5, value=1, step=1, title="Reps", width=200)

select.js_on_change(
    "value", 
    CustomJS(
        args={
            'p1_cds':cds_pr_cal['Y'][1],
            'cds_pr_cal':cds_pr_cal, 
            'slider':slider
        },
        code="""
            console.log('select: value=' + this.value, this.toString())
            console.log(cds_pr_cal[cb_obj.value][slider.value])
            
            p1_cds.data = cds_pr_cal[cb_obj.value][slider.value].data
            p1_cds.change.emit()
        """
    )
)

slider.js_on_change(
    'value', 
    CustomJS(
        args={
            'p1_cds':cds_pr_cal['Y'][1],
            'cds_pr_cal':cds_pr_cal,
            'select':select
        },
        code="""
            console.log('select: value=' + this.value, this.toString())
            console.log(cds_pr_cal[select.value][cb_obj.value])
            p1_cds.data = cds_pr_cal[select.value][cb_obj.value].data
            
            p1_cds.change.emit()
        """
    )
)

show(Column(select, slider, plot_pr_cal['Y'][1]))

In [37]:
wods['2020-11-14']

'<div class="flex items-center w-primary mb2 f5"><svg viewBox="0 0 24 24" fill="currentColor" class="flex-none mr1" style="width: 1.5rem; height: 1.5rem;"><path d="M11.99 2C6.47 2 2 6.48 2 12s4.47 10 9.99 10C17.52 22 22 17.52 22 12S17.52 2 11.99 2zm4.24 16L12 15.45 7.77 18l1.12-4.81-3.73-3.23 4.92-.42L12 5l1.92 4.53 4.92.42-3.73 3.23L16.23 18z"></path></svg>You set a new Personal Record!</div><div class="fw5 f3 mb2"><span class="w-primary b">A. </span>Barbell Bench Press 8-5-3-1-1-1</div><div class="mb2"><div class="f4"><div><div class="flex items-center mb2"><svg viewBox="0 0 24 24" fill="currentColor" class="flex-none mr1 w-green" style="width: 1.5rem; height: 1.5rem;"><path d="M18.7371986,4.61207126 C16.9593812,2.98976475 14.594574,2 12,2 C6.48,2 2,6.48 2,12 C2,17.52 6.48,22 12,22 C17.52,22 22,17.52 22,12 C22,11.3235779 21.9327278,10.6627725 21.8045133,10.0239138 L19.9982233,11.8302039 C19.9994058,11.8866629 20,11.9432636 20,12 C20,16.41 16.41,20 12,20 C7.59,20 4,16.41 4,12 C4,7.59 

In [34]:
show(Div(text=wods['2020-11-14'].replace(
    '<div class="w-gray mr2 tc self-start" style="width: 1.125rem;">', 
    '<div class="w-gray mr2 tc self-start" style="display: inline-block; float:left;">').replace(
    '<span class="">', '<span class="">. '
)))